In [1]:
import os
import reportlab
from reportlab.lib.units import inch
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfgen import canvas
from reportlab.lib import colors
from reportlab.pdfgen.canvas import Color, toColor
from reportlab.lib.pagesizes import letter, A4
from reportlab.platypus import Paragraph, Table, TableStyle
from reportlab.lib.styles import ParagraphStyle, getSampleStyleSheet
from reportlab.graphics import shapes
from reportlab.graphics.charts.textlabels import Label
from reportlab.graphics.charts.lineplots import ScatterPlot
from reportlab.graphics.charts.barcharts import VerticalBarChart
from reportlab.graphics.charts.legends import Legend
import numpy as np
from PIL import Image

In [2]:
pdfmetrics.getRegisteredFontNames()

['Symbol', 'ZapfDingbats']

In [3]:
# To import and use True Type Fonts, we need
from reportlab.pdfbase.ttfonts import TTFont

font_folder = 'C:\\Program Files (x86)\\Microsoft Office\\root\\VFS\Fonts\\private\\' # Or wherever you might have TTF files

pdfmetrics.registerFont(TTFont('Playbill', font_folder + 'PLAYBILL.TTF'))
pdfmetrics.registerFont(TTFont('Rage', font_folder + 'RAGE.TTF'))

pdfmetrics.getRegisteredFontNames()

['Playbill', 'Rage', 'Symbol', 'ZapfDingbats']

In [4]:
r1_font_folder = os.path.dirname(reportlab.__file__) + os.sep + 'fonts'
barcode_39_path = os.path.join(r1_font_folder, 'LibreBarcode39-Regular.ttf')
pdfmetrics.registerFont(TTFont('Barcode39', barcode_39_path))
pdfmetrics.getRegisteredFontNames()

['Barcode39', 'Playbill', 'Rage', 'Symbol', 'ZapfDingbats']

In [5]:
stylesheet = getSampleStyleSheet()
#stylesheet.list())
normalStyle = stylesheet["Normal"]
normalStyle.listAttrs()

name = Normal
parent = None
alignment = 0
allowOrphans = 0
allowWidows = 1
backColor = None
borderColor = None
borderPadding = 0
borderRadius = None
borderWidth = 0
bulletAnchor = start
bulletFontName = Helvetica
bulletFontSize = 10
bulletIndent = 0
embeddedHyphenation = 0
endDots = None
firstLineIndent = 0
fontName = Helvetica
fontSize = 10
hyphenationLang = 
justifyBreaks = 0
justifyLastLine = 0
leading = 12
leftIndent = 0
linkUnderline = 0
rightIndent = 0
spaceAfter = 0
spaceBefore = 0
spaceShrinkage = 0.05
splitLongWords = 1
strikeColor = None
strikeGap = 1
strikeOffset = 0.25*F
strikeWidth = 
textColor = Color(0,0,0,1)
textTransform = None
underlineColor = None
underlineGap = 1
underlineOffset = -0.125*F
underlineWidth = 
uriWasteReduce = 0
wordWrap = None


In [6]:
normalStyle.fontName = 'Rage'
normalStyle.fontsize= 14
normalStyle.listAttrs()

name = Normal
parent = None
alignment = 0
allowOrphans = 0
allowWidows = 1
backColor = None
borderColor = None
borderPadding = 0
borderRadius = None
borderWidth = 0
bulletAnchor = start
bulletFontName = Helvetica
bulletFontSize = 10
bulletIndent = 0
embeddedHyphenation = 0
endDots = None
firstLineIndent = 0
fontName = Rage
fontSize = 10
fontsize = 14
hyphenationLang = 
justifyBreaks = 0
justifyLastLine = 0
leading = 12
leftIndent = 0
linkUnderline = 0
rightIndent = 0
spaceAfter = 0
spaceBefore = 0
spaceShrinkage = 0.05
splitLongWords = 1
strikeColor = None
strikeGap = 1
strikeOffset = 0.25*F
strikeWidth = 
textColor = Color(0,0,0,1)
textTransform = None
underlineColor = None
underlineGap = 1
underlineOffset = -0.125*F
underlineWidth = 
uriWasteReduce = 0
wordWrap = None


## Page 1 of PDF file

In [7]:
img_file = 'SkullsBackground_800x800.png'
im = Image.open(img_file)

c = canvas.Canvas("reportlab_pdf.pdf", pagesize=letter)
width, height = letter
print(f'width={width}, height={height}')

c.ellipse(200, 200, 412, 592, stroke=1, fill=0)
c.drawString(100,300, "Hello World")
c.drawRightString(512, 500, "Goodbye, World")
c.drawCentredString(width/2, height/2, "Heyyyy")
c.setFont('Barcode39', 48)
c.drawString(100, 450, "1234567890")
c.setFont('Playbill', 16)
textobject = c.beginText (50, 642)
text_block = '''Here we are writing some multi-line text in Python and sending it to be printed out to PDF using a textobject.
Since we have line breaks in the
text, these breaks are not
ignored and the text is not run together as if it were one long string. The existing carriage returns and line feeds are respected, and lines of text may extend past the page edge
with the number of lines output equal to the number of lines in this multi-line block.'''
textobject.textLines(text_block)
c.drawText(textobject)

#c.drawImage(img_file, 300, 300, width=200, height=(200*im.height/im.width), mask=[252, 255, 252, 255, 252, 255]) # instead of mask-'auto' this has the result of turning near pure white pixels transparent
c.drawImage(img_file, 300, 300, width=200, height=(200*im.height/im.width), mask=[100, 105, 40, 45, 158, 160]) # For the skulls PNG file, these values capture most of the background for the image, leaving only the skulls and shadows

para_block = '''Here we are writing some multi-line text in Python and sending it to be printed out to PDF using a Paragraph.
Although we have line breaks in the
text, these are presumably
ignored and the text is run together as if it were one long string. If so, then when the carriage returns and line feeds are deleted, we should expect to see words slammed
together where one line meets the other in this multi-line block'''
para = Paragraph(para_block, normalStyle)
w, h = para.wrap(width-100, height-600)
para.drawOn(c, 50, height-600)

c.showPage () # Creates the page - all other drawing on the canvas will go onto the next page until another showPage() is executed

width=612.0, height=792.0


## Page 2 of PDF file

In [8]:
header = [f"Col {i+1}" for i in range(8)]
data = list(np.random.uniform(0.0, 1.0, (24, 8)).round(4).astype(str))
data = [list(r) for r in data]
data = [header] + data
tbl = Table(data)
LIST_STYLE = TableStyle([
      ('TEXTCOLOR', (0,0), (-1,0), colors.darkblue)
    , ('BACKGROUND', (0,0), (-1,0), colors.grey)
])
for row in range (1, 25):
    for col in range(8):
        LIST_STYLE.add('BACKGROUND', (col, row), (col, row), Color(1.0 - float(data[row][col]), float(data[row][col]), 0.0))
        #LIST_STYLE. add('BACKGROUND', (col, row), (col, row), toColor(f'rgb({int(255 * (1.0 - float (data[row][col])))}, {int (255 * float (data[row][col]))}, 0)'))
        #if float (data[row][col]) ›= 0.5:
        #   LIST_STYLE.add('BACKGROUND', (col, row), (col, row), colors.green)

tbl.setStyle(LIST_STYLE)
w, h = tbl.wrapOn(c, 0, 0)
tbl.drawOn(c, 0.5*inch, 0.5*inch)
c.showPage()

## Page 3 of PDF file

In [9]:
d = shapes.Drawing(width, height)
d.add(shapes.Circle(200, 100, 50, fillColor=Color(1.0, 1.0, 0.0))) # Yellow = rgb (255, 255, 0)

lab = Label()
lab.setOrigin(100, 90)
lab.boxAnchor = 'ne'
lab.angle = 45
lab.dx = 0
lab.dy = -20
lab.boxStrokeColor = colors.green
lab.setText('Some\nMulti-Line\nLabel')
d.add(lab)

# See also https://www.reportlab.com/chartgallery/
dta = [tuple(np.random.normal(20, 5, 400))]
scat = ScatterPlot()
scat.x = 50
scat.y = 500
scat.width = width - 100
scat.height = height - 550
#scat.data = [(str(x), x, y) for x, y in enumerate(sorted(dta[0]))]
scat.data = [[(x, y)] for x, y in enumerate(sorted(dta[0]))] # example: [[(2.25, 4.48)], [(3.59. 4.42)], [(3.52, 3.58)]]
d.add(scat)

hist_dta = np.histogram(dta[0], bins=20)
bc = VerticalBarChart()
bc.x = 50
bc.y = 200
scat.width = width - 100
scat.height = 250
bc.data = [tuple(hist_dta[0])]
d.add(bc)

d.drawOn(c, 0, 0)
c.showPage()

In [10]:
c.save() # Finalizes the output by saving the PDF file and all pages that were created]